In [7]:
!conda install -c conda-forge geocoder --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00   1.30 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00   8.48 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  35.40 MB/s


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

wiki_source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

wiki = BeautifulSoup(wiki_source, 'lxml')

wiki_table = wiki.find('table')

m_postal_codes = pd.read_html(str(wiki_table), header=0)[0]

In order to work around a limit on the API call, I moved this code locally and then just copy and pasted the latitude / longitude list here

import geocoder
from bs4 import BeautifulSoup
import requests
import pandas as pd

wiki_source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

wiki = BeautifulSoup(wiki_source, 'lxml')

wiki_table = wiki.find('table')

m_postal_codes = pd.read_html(str(wiki_table), header=0)[0]

#cleaning the codes marked as Not Assigned
toronto_pcodes = m_postal_codes[m_postal_codes['Borough'] != 'Not assigned']
toronto_pcodes = m_postal_codes[m_postal_codes['Neighbourhood'] != 'Not assigned']

#use a groupby to concatenate all the neighbourhood values to a single row
pcode_grouped = toronto_pcodes.groupby([toronto_pcodes['Postcode'], toronto_pcodes['Borough']])['Neighbourhood'].apply(', '.join).reset_index()
pcode_grouped.head()

#I found some stray brackets in the website data so I'm cleaning them here
pcode_grouped['Neighbourhood'].replace(to_replace=']', value='', regex=True, inplace=True)
pcode_grouped['Neighbourhood'].replace(to_replace='\[', value='', regex=True, inplace=True)

def gcoder_lookup (postcode):
    lat_lng_list = []
    for code in postcode:
        # initialize your variable to None
        lat_lng_coords = None

        # loop until you get the coordinates
        while(lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(code))
            lat_lng_coords = g.latlng
        lat_lng_list.append(lat_lng_coords[1])
    return lat_lng_list

#test_list = [gcode_grouped['Neighbourhood']]

test_ll_list = gcoder_lookup(list(pcode_grouped['Postcode']))

print(test_ll_list)

In [2]:
#cleaning the codes marked as Not Assigned
toronto_pcodes = m_postal_codes[m_postal_codes['Borough'] != 'Not assigned']
toronto_pcodes = m_postal_codes[m_postal_codes['Neighbourhood'] != 'Not assigned']

#use a groupby to concatenate all the neighbourhood values to a single row
pcode_grouped = toronto_pcodes.groupby([toronto_pcodes['Postcode'], toronto_pcodes['Borough']])['Neighbourhood'].apply(', '.join).reset_index()
pcode_grouped.head()

#I found some stray brackets in the website data so I'm cleaning them here
pcode_grouped['Neighbourhood'].replace(to_replace=']', value='', regex=True, inplace=True)
pcode_grouped['Neighbourhood'].replace(to_replace='\[', value='', regex=True, inplace=True)

In [11]:
pcode_grouped['Latitude'] = [43.8066863, 43.78453510000001, 43.7635726, 43.7709921, 43.773136, 43.7447342, 43.7279292, 43.7111117, 43.716316, 43.692657, 43.7574096, 43.7500715, 43.7942003, 43.7816375, 43.8152522, 43.7995252, 43.8361247, 43.8037622, 43.7785175, 43.7869473, 43.7574902, 43.789053, 43.7701199, 43.7527583, 43.7827364, 43.7532586, 43.7459058, 43.7258997, 43.7543283, 43.7679803, 43.7374732, 43.7390146, 43.7284964, 43.7616313, 43.72588229999999, 43.7063972, 43.6953439, 43.6763574, 43.7090604, 43.7053689, 43.685347, 43.6795571, 43.6689985, 43.6595255, 43.7280205, 43.7127511, 43.7153834, 43.7043244, 43.6895743, 43.6864123, 43.6795626, 43.667967, 43.6658599, 43.6542599, 43.6571618, 43.6514939, 43.6447708, 43.6579524, 43.65057119999999, 43.6408157, 43.6471768, 43.6481985, 43.7332825, 43.7116948, 43.6969476, 43.6727097, 43.6626956, 43.6532057, 43.6289467, 43.6464352, 43.6484292, 43.718518, 43.709577, 43.6937813, 43.68902560000001, 43.669542, 43.6690051, 43.6479267, 43.6368472, 43.7137562, 43.6911158, 43.6731853, 43.6616083, 43.6489597, 43.6515706, 43.6369656, 43.6627439, 43.6056466, 43.6024137, 43.6536536, 43.6362579, 43.6288408, 43.6678556, 43.65094320000001, 43.6435152, 43.7563033, 43.7247659, 43.706876, 43.696319, 43.6889054, 43.7394164, 43.7067483]

In [12]:
pcode_grouped['Longitude'] = [-79.1943534, -79.1604971, -79.1887115, -79.2169174, -79.23947609999999, -79.23947609999999, -79.2620294, -79.2845772, -79.23947609999999, -79.2648481, -79.273304, -79.2958491, -79.2620294, -79.3043021, -79.2845772, -79.3183887, -79.20563609999999, -79.3634517, -79.3465557, -79.385975, -79.37471409999999, -79.40849279999999, -79.40849279999999, -79.40004929999999, -79.4422593, -79.3296565, -79.352188, -79.34092299999999, -79.4422593, -79.4872619, -79.4647633, -79.5069436, -79.4956974, -79.5209994, -79.3155716, -79.30993699999999, -79.3183887, -79.2930312, -79.3634517, -79.3493719, -79.3381065, -79.352188, -79.3155716, -79.34092299999999, -79.3887901, -79.3901975, -79.4056784, -79.3887901, -79.3831599, -79.40004929999999, -79.3775294, -79.3676753, -79.3831599, -79.36063589999999, -79.3789371, -79.3754179, -79.37330639999999, -79.3873826, -79.3845675, -79.3817523, -79.3815764, -79.3798169, -79.4197497, -79.4169356, -79.4113072, -79.4056784, -79.40004929999999, -79.40004929999999, -79.3944199, -79.37484599999999, -79.3822802, -79.4647633, -79.4450726, -79.4281914, -79.453512, -79.4225637, -79.4422593, -79.4197497, -79.4281914, -79.49007379999999, -79.47601329999999, -79.4872619, -79.4647633, -79.45632499999999, -79.4844499, -79.615819, -79.321558, -79.5013207, -79.5434841, -79.5069436, -79.49850909999999, -79.5209994, -79.5322424, -79.5547244, -79.5772008, -79.56596329999999, -79.5322424, -79.5181884, -79.5322424, -79.5547244, -79.58843689999999, -79.59405439999999]

In [13]:
pcode_grouped.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
